### In preprocessing 2, we preprocessed movie data upto 2017. Now using wikipead and TMDB we'll scrape, extract & add movie data of 2018, 2019,2020,2021 to the datset created in preprocessing 2 i.e. "movie_data_upto_2017.csv", thereby creating new dataset consisting movie data upto 2021.

In [1]:
import pandas as pd
import numpy as np

#### Extracting some features of 2018 movies from Wikipedia

In [2]:
# extracting tables from wikipedia page
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
#  the argument 2,3,4,5 here indicates the table number
t1 = pd.read_html(link, header=0)[2]
t2 = pd.read_html(link, header=0)[3]
t3 = pd.read_html(link, header=0)[4]
t4 = pd.read_html(link, header=0)[5]

In [3]:
table = t1.append(t2.append(t3.append(t4, ignore_index = True), ignore_index = True),ignore_index = True)

In [4]:
table

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN
...,...,...,...,...,...,...,...
267,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[164]
268,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[137]
269,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[228]
270,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",NaN,[260]


In [5]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = "4bfa18d4dd4f5d041a3d88cfb6728654"

In [6]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def extract_genres(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
        # This "if not result" condition is specifically added for 2020 or after data as for some movie we don't get result on TMBb
        return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        # extracting data in json format
        data_json = response.json()
        if data_json['genres']:
        # Actually there's a key "genres" which basically holds a list of dictionaries and each dictionary has one genre
        # paired to key "name". So we here loop thorugh each dictionary and extract each & every genre of a given movie.    
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN

In [7]:
table['genres'] = table['Title'].map(lambda x: extract_genres(str(x)))

In [8]:
table.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.,genres
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN,Horror Mystery Thriller
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN,Thriller Drama
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN,Action Thriller
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN,Drama History Western
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN,Action Thriller Mystery


In [9]:
movie_data_2018 = table[["Title", "Cast and crew", "genres"]]

In [10]:
movie_data_2018

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery
...,...,...,...
267,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime
268,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure
269,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History
270,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action


In [11]:
def extract_director_name(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (director/screenplay)" in x:
        return x.split(" (director/screenplay)")[0]
    else:
        return x.split(" (directors)")[0]

In [12]:
movie_data_2018["director_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_director_name(x))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\398042792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2018["director_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_director_name(x))


In [13]:
movie_data_2018

,Title,Cast and crew,genres,director_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra
...,...,...,...,...
267,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen
268,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay
269,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder
270,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama


In [14]:
def extract_actor_names(x, actor_num):
    # here we are separating actor names from rest of the crew.    
    cast = x.split("screenplay); ")[-1]
    if actor_num == 0:
        return cast.split(", ")[actor_num]
    elif actor_num == 1:
        if len(cast.split(", ")) < 2:
            return np.NaN
        else:
            return cast.split(", ")[actor_num]
    elif actor_num == 2:
        if len(cast.split(", ")) < 3:
            return np.NaN
        else:
            return cast.split(", ")[actor_num]
    

In [15]:
movie_data_2018["actor_1_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_actor_names(x, 0))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\4145939486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2018["actor_1_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_actor_names(x, 0))


In [16]:
movie_data_2018

,Title,Cast and crew,genres,director_name,actor_1_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton,Bryan Brown
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson
...,...,...,...,...,...
267,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell
268,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale
269,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones
270,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman


In [17]:
movie_data_2018["actor_2_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_actor_names(x, 1))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\1023051770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2018["actor_2_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_actor_names(x, 1))


In [18]:
movie_data_2018

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga
...,...,...,...,...,...,...
267,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly
268,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams
269,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones,Armie Hammer
270,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan


In [19]:
movie_data_2018["actor_3_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_actor_names(x, 2))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\2383325369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2018["actor_3_name"] = movie_data_2018["Cast and crew"].map(lambda x: extract_actor_names(x, 2))


In [20]:
movie_data_2018

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
...,...,...,...,...,...,...,...
267,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
268,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell
269,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux
270,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell


In [21]:
movie_data_2018 = movie_data_2018.rename(columns={"Title": "movie_title"})

In [22]:
new_movie_data_2018 = movie_data_2018.loc[:, ["movie_title", "genres", "director_name", "actor_1_name", "actor_2_name", "actor_3_name"]]

In [23]:
new_movie_data_2018

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
...,...,...,...,...,...,...
267,Holmes & Watson,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
268,Vice,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell
269,On the Basis of Sex,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux
270,Destroyer,Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell


In [24]:
new_movie_data_2018["actor_2_name"] = new_movie_data_2018["actor_2_name"].replace(np.NaN, "unknown")
new_movie_data_2018["actor_3_name"] = new_movie_data_2018["actor_3_name"].replace(np.NaN, "unknown")

In [25]:
new_movie_data_2018["movie_title"] = new_movie_data_2018["movie_title"].str.lower()

In [26]:
new_movie_data_2018

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,unknown
4,the commuter,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
...,...,...,...,...,...,...
267,holmes & watson,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
268,vice,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell
269,on the basis of sex,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux
270,destroyer,Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell


In [27]:
new_movie_data_2018["combine"] = new_movie_data_2018["genres"] +" "+ new_movie_data_2018["director_name"] +" "+ new_movie_data_2018["actor_1_name"] +" "+ new_movie_data_2018["actor_2_name"] +" "+new_movie_data_2018["actor_3_name"]

In [28]:
new_movie_data_2018

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,combine
0,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller Adam Robitel Lin Shaye...
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama Lauren Wolkstein Alex Pettyfer ...
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller Simon West Dominic Cooper Aust...
3,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western Warwick Thornton Bryan B...
4,the commuter,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery Jaume Collet-Serra Lia...
...,...,...,...,...,...,...,...
267,holmes & watson,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Mystery Adventure Comedy Crime Etan Cohen Will...
268,vice,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Thriller Science Fiction Action Adventure Adam...
269,on the basis of sex,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Drama History Mimi Leder Felicity Jones Armie ...
270,destroyer,Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Thriller Crime Drama Action Karyn Kusama Nicol...


#### Extracting some features of 2019 movies from Wikipedia

#### The steps will be similar to what we did with 2018 data

In [29]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
#  the argument 2,3,4,5 here indicates the table number
t1_2019 = pd.read_html(link, header=0)[2]
t2_2019 = pd.read_html(link, header=0)[3]
t3_2019 = pd.read_html(link, header=0)[4]
t4_2019 = pd.read_html(link, header=0)[5]

In [30]:
table_2019 = t1_2019.append(t2_2019.append(t3_2019.append(t4_2019, ignore_index = True), ignore_index = True),ignore_index = True)

In [31]:
table_2019

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
237,DECEMBER,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
238,DECEMBER,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[221]
239,DECEMBER,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[222]
240,DECEMBER,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[223]


In [32]:
table_2019['genres'] = table_2019['Title'].map(lambda x: extract_genres(str(x)))

In [33]:
movie_data_2019 = table_2019[["Title", "Cast and crew", "genres"]]

In [34]:
movie_data_2019

,Title,Cast and crew,genres
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama
...,...,...,...
237,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family
238,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance
239,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Drama Action Thriller
240,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History


In [35]:
movie_data_2019["director_name"] = movie_data_2019["Cast and crew"].map(lambda x: extract_director_name(x))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\1780834350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2019["director_name"] = movie_data_2019["Cast and crew"].map(lambda x: extract_director_name(x))


In [36]:
movie_data_2019["actor_1_name"] = movie_data_2019["Cast and crew"].map(lambda x: extract_actor_names(x, 0))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\677343466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2019["actor_1_name"] = movie_data_2019["Cast and crew"].map(lambda x: extract_actor_names(x, 0))


In [37]:
movie_data_2019["actor_2_name"] = movie_data_2019["Cast and crew"].map(lambda x: extract_actor_names(x, 1))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\7704029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2019["actor_2_name"] = movie_data_2019["Cast and crew"].map(lambda x: extract_actor_names(x, 1))


In [38]:
movie_data_2019["actor_3_name"] = movie_data_2019["Cast and crew"].map(lambda x: extract_actor_names(x, 2))

In [39]:
movie_data_2019 = movie_data_2019.rename(columns={"Title": "movie_title"})

In [40]:
new_movie_data_2019 = movie_data_2019.loc[:, ["movie_title", "genres", "director_name", "actor_1_name", "actor_2_name", "actor_3_name"]]

In [41]:
new_movie_data_2019["actor_2_name"] = new_movie_data_2019["actor_2_name"].replace(np.NaN, "unknown")
new_movie_data_2019["actor_3_name"] = new_movie_data_2019["actor_3_name"].replace(np.NaN, "unknown")

In [42]:
new_movie_data_2019["movie_title"] = new_movie_data_2019["movie_title"].str.lower()

In [43]:
new_movie_data_2019["combine"] = new_movie_data_2019["genres"] +" "+ new_movie_data_2019["director_name"] +" "+ new_movie_data_2019["actor_1_name"] +" "+ new_movie_data_2019["actor_2_name"] +" "+new_movie_data_2019["actor_3_name"]

In [44]:
new_movie_data_2019

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,combine
0,escape room,Horror Thriller Mystery,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Horror Thriller Mystery Adam Robitel Taylor Ru...
1,rust creek,Thriller Drama,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Thriller Drama Jen McGowan Hermione Corfield J...
2,american hangman,Thriller,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Thriller Wilson Coneybeare Donald Sutherland V...
3,a dog's way home,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Drama Adventure Family Charles Martin Smith Br...
4,the upside,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Comedy Drama Neil Burger Bryan Cranston Kevin ...
...,...,...,...,...,...,...,...
237,spies in disguise,Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family Nick ...
238,little women,Drama Romance,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance Greta Gerwig Saoirse Ronan Emma ...
239,1917,War Drama Action Thriller,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action Thriller Sam Mendes George Ma...
240,just mercy,Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History Destin Daniel Cretton Mich...


#### Extracting some features of 2020 movies from Wikipedia (steps are same again)

In [45]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
#  the argument 2,3,4,5 here indicates the table number
t1_2020 = pd.read_html(link, header=0)[2]
t2_2020 = pd.read_html(link, header=0)[3]
t3_2020 = pd.read_html(link, header=0)[4]
t4_2020 = pd.read_html(link, header=0)[5]

In [46]:
table_2020 = t1_2020.append(t2_2020.append(t3_2020.append(t4_2020, ignore_index = True), ignore_index = True),ignore_index = True)

In [47]:
table_2020['genres'] = table_2020['Title'].map(lambda x: extract_genres(str(x)))

In [48]:
movie_data_2020 = table_2020[["Title", "Cast and crew", "genres"]]

In [49]:
movie_data_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [50]:
movie_data_2020["director_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_director_name(x))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\2163217486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2020["director_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_director_name(x))


In [51]:
movie_data_2020["actor_1_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_actor_names(x, 0))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\2551291283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2020["actor_1_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_actor_names(x, 0))


In [52]:
movie_data_2020["actor_2_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_actor_names(x, 1))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\320297937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2020["actor_2_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_actor_names(x, 1))


In [53]:
movie_data_2020["actor_3_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_actor_names(x, 2))

C:\Users\pranj\AppData\Local\Temp\ipykernel_6172\3145625747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data_2020["actor_3_name"] = movie_data_2020["Cast and crew"].map(lambda x: extract_actor_names(x, 2))


In [54]:
movie_data_2020 = movie_data_2020.rename(columns={"Title": "movie_title"})

In [55]:
new_movie_data_2020 = movie_data_2020.loc[:, ["movie_title", "genres", "director_name", "actor_1_name", "actor_2_name", "actor_3_name"]]

In [56]:
new_movie_data_2020["actor_2_name"] = new_movie_data_2020["actor_2_name"].replace(np.NaN, "unknown")
new_movie_data_2020["actor_3_name"] = new_movie_data_2020["actor_3_name"].replace(np.NaN, "unknown")

In [57]:
new_movie_data_2020["movie_title"] = new_movie_data_2020["movie_title"].str.lower()

In [58]:
new_movie_data_2020["combine"] = new_movie_data_2020["genres"] +" "+ new_movie_data_2020["director_name"] +" "+ new_movie_data_2020["actor_1_name"] +" "+ new_movie_data_2020["actor_2_name"] +" "+new_movie_data_2020["actor_3_name"]

In [59]:
new_movie_data_2020

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,combine
0,the grudge,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller Nicolas Pesce Andrea R...
1,underwater,Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller William...
2,like a boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy Miguel Arteta Tiffany Haddish Rose Byrn...
3,three christs,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama Jon Avnet Richard Gere Peter Dinklage Wa...
4,inherit the viper,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama Anthony Jerjen Josh Hartn...
...,...,...,...,...,...,...,...
269,we can be heroes,Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy Robert Rodriguez ...
270,news of the world,Drama Western Adventure,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Paul Greengrass Tom Ha...
271,one night in miami...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama Regina King Kingsley Ben-Adir Eli Goree ...
272,promising young woman,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama Emerald Fennell Carey Mul...


#### Extracting some features of 2021 movies from Wikipedia (steps are same again)

In [60]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
#  the argument 2,3,4,5 here indicates the table number
t1_2021 = pd.read_html(link, header=0)[2]
t2_2021 = pd.read_html(link, header=0)[3]
t3_2021 = pd.read_html(link, header=0)[4]
t4_2021 = pd.read_html(link, header=0)[5]

In [61]:
table_2021 = t1_2021.append(t2_2021.append(t3_2021.append(t4_2021, ignore_index = True), ignore_index = True),ignore_index = True)

In [62]:
table_2021['genres'] = table_2021['Title'].map(lambda x: extract_genres(str(x)))

In [63]:
movie_data_2021 = table_2021[["Title", "Cast and crew", "genres"]]

In [64]:
# We have to remove this null record as when extracting director name it will show error as on of our "Cast and crew" cell 
# is empty
movie_data_2021.isna().sum()

Title            1
Cast and crew    1
genres           1
dtype: int64

In [65]:
movie_data_2021 = movie_data_2021.dropna(how='any')

In [66]:
movie_data_2021.isna().sum()

Title            0
Cast and crew    0
genres           0
dtype: int64

In [67]:
movie_data_2021["director_name"] = movie_data_2021["Cast and crew"].map(lambda x: extract_director_name(x))

In [68]:
movie_data_2021["actor_1_name"] = movie_data_2021["Cast and crew"].map(lambda x: extract_actor_names(x, 0))

In [69]:
movie_data_2021["actor_2_name"] = movie_data_2021["Cast and crew"].map(lambda x: extract_actor_names(x, 1))

In [70]:
movie_data_2021["actor_3_name"] = movie_data_2021["Cast and crew"].map(lambda x: extract_actor_names(x, 2))

In [71]:
movie_data_2021 = movie_data_2021.rename(columns={"Title": "movie_title"})

In [72]:
new_movie_data_2021 = movie_data_2021.loc[:, ["movie_title", "genres", "director_name", "actor_1_name", "actor_2_name", "actor_3_name"]]

In [73]:
new_movie_data_2021["actor_2_name"] = new_movie_data_2021["actor_2_name"].replace(np.NaN, "unknown")
new_movie_data_2021["actor_3_name"] = new_movie_data_2021["actor_3_name"].replace(np.NaN, "unknown")

In [74]:
new_movie_data_2021["movie_title"] = new_movie_data_2021["movie_title"].str.lower()

In [75]:
new_movie_data_2021["combine"] = new_movie_data_2021["genres"] +" "+ new_movie_data_2021["director_name"] +" "+ new_movie_data_2021["actor_1_name"] +" "+ new_movie_data_2021["actor_2_name"] +" "+new_movie_data_2021["actor_3_name"]

In [76]:
new_movie_data_2021

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,combine
0,shadow in the cloud,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War Roseanne Liang Chloë Grace M...
1,the white tiger,Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama Ramin Bahrani Adarsh Gourav Rajkummar Ra...
2,locked down,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama Doug Liman Anne Hathaway Ch...
3,the dig,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History Simon Stone Carey Mulligan Ralph...
4,outside the wire,Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction Mikael Håfströ...
...,...,...,...,...,...,...,...
352,the king's man,Action Adventure Thriller War Mystery,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery Matthew ...
353,the tragedy of macbeth,Drama War,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War Joel Coen Denzel Washington Frances ...
354,a journal for jordan,Drama Romance,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance Denzel Washington Michael B. Jor...
355,american underdog,Drama,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama Erwin brothers Zachary Levi Anna Paquin ...


In [77]:
movie_data_upto_2017 = pd.read_csv("movie_data_upto_2017.csv")

In [78]:
movie_data_upto_2017

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,combine
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi James Cameron ...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy Gore Verbinski Johnny...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller Sam Mendes Christoph...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller Christopher Nolan Tom Hardy Ch...
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary Doug Walker Doug Walker Rob Walker...
...,...,...,...,...,...,...,...
5359,the incredible jessica james,Romance Comedy,Jim Strouse,Jessica Williams,Chris O'Dowd,Keith Stanfield,Romance Comedy Jim Strouse Jessica Williams Ch...
5360,can't buy my love,Romance,Farhad Mann,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,Romance Farhad Mann Adelaide Kane Benjamin Hol...
5361,thick lashes of lauri mäntyvaara,Romance Comedy,Hannaleena Hauru,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Romance Comedy Hannaleena Hauru Inka Haapamäki...
5362,cop and a half: new recruit,Crime Comedy Action Family,Jonathan A. Rosenbaum,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Crime Comedy Action Family Jonathan A. Rosenba...


In [79]:
movie_data_upto_2021 = movie_data_upto_2017.append(new_movie_data_2018.append(new_movie_data_2019.append(new_movie_data_2020.append(new_movie_data_2021, ignore_index = True), ignore_index = True), ignore_index = True),ignore_index = True)

In [80]:
movie_data_upto_2021

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,combine
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi James Cameron ...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy Gore Verbinski Johnny...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller Sam Mendes Christoph...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller Christopher Nolan Tom Hardy Ch...
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary Doug Walker Doug Walker Rob Walker...
...,...,...,...,...,...,...,...
6503,the king's man,Action Adventure Thriller War Mystery,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery Matthew ...
6504,the tragedy of macbeth,Drama War,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War Joel Coen Denzel Washington Frances ...
6505,a journal for jordan,Drama Romance,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance Denzel Washington Michael B. Jor...
6506,american underdog,Drama,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama Erwin brothers Zachary Levi Anna Paquin ...


In [81]:
movie_data_upto_2021.isna().sum()

movie_title      0
genres           3
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
combine          3
dtype: int64

In [82]:
movie_data_upto_2021 = movie_data_upto_2021.dropna(how='any')

In [83]:
movie_data_upto_2021.isna().sum()

movie_title      0
genres           0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
combine          0
dtype: int64

In [84]:
movie_data_upto_2021

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,combine
0,avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi James Cameron ...
1,pirates of the caribbean: at world's end,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy Gore Verbinski Johnny...
2,spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller Sam Mendes Christoph...
3,the dark knight rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller Christopher Nolan Tom Hardy Ch...
4,star wars: episode vii - the force awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary Doug Walker Doug Walker Rob Walker...
...,...,...,...,...,...,...,...
6503,the king's man,Action Adventure Thriller War Mystery,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery Matthew ...
6504,the tragedy of macbeth,Drama War,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War Joel Coen Denzel Washington Frances ...
6505,a journal for jordan,Drama Romance,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance Denzel Washington Michael B. Jor...
6506,american underdog,Drama,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama Erwin brothers Zachary Levi Anna Paquin ...


In [85]:
movie_data_upto_2021.to_csv("movie_data_upto_2021.csv", index=False)